## Model Deployment
In the previous chapters we focussed on creating predicton models. Jupyter notebooks are great in data exploration, transformation and cleaning and in model tuning and evaluation. However, once you are fine with your model is time to use is in practice. Therefore we save the model to a file so we can use it over and over again.  We illustrate this using the customer churn case. 

In [1]:
# we first build the model
import pandas as pd
telecom = pd.read_csv('data/telecom_case.csv',sep=',')
telecom = telecom.drop(['servcon','acq_costs'],axis=1)
telecom = telecom.dropna()
telecom = pd.get_dummies(telecom,columns=['gender'])
from sklearn.model_selection import train_test_split
X = telecom.drop('switch',axis=1)
y = telecom['switch']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30)
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [2]:
# we now save the model to a file
# see https://scikit-learn.org/stable/modules/model_persistence.html
from joblib import dump
dump(model, '../models/churn_prediction_model.joblib') 

['../models/churn_prediction_model.joblib']

In [3]:
# We will now use this model to predict wether or not some current customers might leave or not.
# This will typically be part of a end-user application and run e.g. weekly on the customer database

def PredictSwitch(model,no_months,age,income,no_months_ctr,no_months_ctr_end,phones,age_head_hh,edu_head_hh,
                  income_head_hh,house_type,work_situation,no_calls,no_minutes,no_msgs,
                  no_calls6,no_minutes6,no_msgs6,gender):
    import pandas as pd
    customer=pd.DataFrame(columns=['no_months','age','income','no_months_ctr','no_months_ctr_end','phones',
                    'age_head_hh','edu_head_hh','income_head_hh','house_type','work_situation',
                    'no_calls','no_minutes','no_msgs','no_calls6','no_minutes6','no_msgs6','gender'])

    new_customer = {'no_months':no_months,
                    'age':age,
                    'income':income,
                    'no_months_ctr':no_months_ctr,
                    'no_months_ctr_end':no_months_ctr_end,
                    'phones':phones,
                    'age_head_hh':age_head_hh,
                    'edu_head_hh':edu_head_hh,
                    'income_head_hh':income_head_hh,
                    'house_type':house_type,
                    'work_situation':work_situation,
                    'no_calls':no_calls,
                    'no_minutes':no_minutes,
                    'no_msgs':no_msgs,
                    'no_calls6':no_calls6,
                    'no_minutes6':no_minutes6,
                    'no_msgs6':no_msgs6,
                    'gender':gender}
    
    customer = customer.append(new_customer,ignore_index=True)

    if gender == 'M':
        customer['gender_M'] = 1
        customer['gender_V'] = 0
    else:
        customer['gender_M'] = 0
        customer['gender_V'] = 1        
    customer.drop(columns=['gender'],axis=1,inplace=True)

    # we can't use pd.get_dummies here because not all values (M,V) are available
    
    switch = model.predict(customer)
    
    # most sklearn algorithms also offer a predict_proba method that returns an array of probabilities per class:
    switch_proba = model.predict_proba(customer)
    return switch[0],switch_proba[0].max()


from joblib import load
model = load('../models/churn_prediction_model.joblib')

switch = PredictSwitch(model,no_months=28,age=21,income=531.35,no_months_ctr=17,no_months_ctr_end=15,phones=1,
                       age_head_hh=3,
                       edu_head_hh=2,income_head_hh=1,house_type=5,work_situation=1,no_calls=0,no_minutes=0,
                       no_msgs=0,no_calls6=106,no_minutes6=98.02,no_msgs6=15,gender='M')

print(switch)

switch = PredictSwitch(model,no_months=24,age=46,income=300,no_months_ctr=20,no_months_ctr_end=7,phones=2,age_head_hh=46,
                       edu_head_hh=2,income_head_hh=4,house_type=5,work_situation=1,no_calls=50,no_minutes=400,
                       no_msgs=5,no_calls6=70,no_minutes6=8,no_msgs6=10,gender='M')

print(switch)



(1, 1.0)
(0, 0.93)
